In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pycountry_convert as pc
import folium as f
import keras as k
import plotly 

Using TensorFlow backend.


In [35]:
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [36]:
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [37]:
df_covid19 = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv")

In [38]:
df_table = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_time.csv",parse_dates=['Last_Update'])

In [39]:
df_confirmed = df_confirmed.rename(columns={'Country/Region': 'Country', 'Province/State': 'State'})
df_deaths = df_deaths.rename(columns={'Country/Region': 'Country', 'Province/State': 'State'})

df_covid19["Active"] = df_covid19["Confirmed"]-df_covid19["Recovered"]-df_covid19["Deaths"]
df_covid19 = df_covid19.rename(columns={'Country_Region': 'Country'})
df_covid19 = df_covid19.drop(['People_Tested', 'People_Hospitalized'], axis=1)
df_covid19.loc[df_covid19['Country'] == "Taiwan*", "Country"] = "Taiwan"

In [40]:
df_confirmed = df_confirmed.replace(np.nan, '', regex=True)
df_deaths = df_deaths.replace(np.nan, '', regex=True)

In [41]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [42]:
# getting all countries
countries = np.asarray(df_confirmed["Country"])
countries1 = np.asarray(df_covid19["Country"])
# Continent_code to Continent_names
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU' : 'Europe',
    'na' : 'Others'
}

def country_to_continent_code(country):
    try:
        return continents[pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(country))]
    except: 
        return 'na'
    
df_confirmed.insert(2, 'Continent', [country_to_continent_code(country) for country in countries[:]])
df_deaths.insert(2, 'Continent', [country_to_continent_code(country) for country in countries[:]])
df_covid19.insert(1, 'Continent', [country_to_continent_code(country) for country in countries1[:]])

In [48]:
df_confirmed = df_confirmed.replace(np.nan, '', regex=True)
df_deaths = df_deaths.replace(np.nan, '', regex=True)

In [52]:
df_countries_cases = df_covid19.copy().drop(['Lat', 'Long_', 'Continent', 'Last_Update'], axis=1)
df_countries_cases.index = df_covid19['Country']
df_countries_cases = df_countries_cases.drop(['Country'], axis=1)

In [55]:
df_continent_cases = df_covid19.copy().drop(['Lat', 'Long_', 'Country', 'Last_Update'], axis=1)
df_continent_cases = df_continent_cases.groupby(['Continent']).sum()

In [56]:
df_continent_cases

,Confirmed,Deaths,Recovered,Active,Incident_Rate,Mortality_Rate,UID
Continent,,,,,,,
Africa,38563,1638,12862,24063,348.315693,189.164169,23596
Asia,526669,18841,268740,239088,2253.562360,106.944724,18044
Australia,8292,113,7073,1106,59.526460,2.718280,1430
Europe,1383189,137474,494805,750910,9667.056448,232.949163,18538
North America,96278,6235,38820,51223,630.005714,125.755380,7452
South America,190608,9507,63804,117297,475.484749,59.540090,4708
na,1119179,65419,174881,878879,1780.287019,49.883192,23335


In [57]:
df_countries_cases.fillna(0,inplace=True)
df_continent_cases.fillna(0,inplace=True)